In [1]:
import time
import tensorflow as tf
import psutil

class PowerMonitor:
    def __init__(self):
        self.gpu_available = tf.config.list_physical_devices('GPU')
        
        # Hardware power specifications (adjust these values for your system)
        self.cpu_tdp = 65    # Typical TDP for desktop CPUs in watts
        self.gpu_tdp = 250   # Typical TDP for desktop GPUs in watts
        
    def get_stats(self):
        """Get system stats with power estimation"""
        stats = {
            'timestamp': time.time(),
            'cpu_%': psutil.cpu_percent(interval=0.1),
            'ram_mb': psutil.virtual_memory().used / (1024**2),
            'gpu_mem_mb': 0,
            'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85  # Base CPU power
        }
        
        if self.gpu_available:
            try:
                # TensorFlow GPU memory monitoring
                mem_info = tf.config.experimental.get_memory_info('GPU:0')
                stats.update({
                    'gpu_mem_mb': mem_info['current'] / (1024**2),
                    'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85 + 
                              self.gpu_tdp * 0.5 * 0.75  # Add GPU power estimate
                })
            except:
                pass
                
        return stats

# Initialize monitor
monitor = PowerMonitor()

# Celeb

In [2]:
print("=== DATA LOADING ===")
start = monitor.get_stats()
import h5py

# Open the HDF5 file for reading
#with h5py.File("D:\\thesis\dataset\deepfake dataset\images_celeb_224R_processed.h5", "r") as h5f:
with h5py.File("D:\\thesis\dataset\Celeb-Df-v2\images_celeb_balanced_224R_processed.h5", "r") as h5f:
    # Load HOG features
    real_frames_array1 = h5f["ori_actor"][:]
    fake_frames_array1 = h5f["ori_youtube"][:]
import numpy as np
import cv2

# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images


# Resize the images
real_frames_array1 = resize_images(real_frames_array1, target_size=(160, 160))
fake_frames_array1 = resize_images(fake_frames_array1, target_size=(160, 160))

# Checking the new shapes
real_frames_array1.shape, fake_frames_array1.shape


# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===

=== RESOURCE USAGE ===
CPU Usage: 7.5%
RAM Used: 2094.0 MB
Time Usage: 6.9 s
GPU Memory Used: 0.0 MB
Power Consumption: 93W


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Split the data into train (70%) and test (30%)
X_train_real, X_test_real = train_test_split(real_frames_array1, test_size=0.3, random_state=42)
X_train_fake, X_test_fake = train_test_split(fake_frames_array1, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data real shape:", X_train_real.shape)
print("Testing real data shape:", X_test_real.shape)
print("Training fake data shape:", X_train_fake.shape)
print("Testing  fake data shape:", X_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_real, val_hog_real = extract_validation(X_train_real)
train_hog_fake, val_hog_fake = extract_validation(X_train_fake)

# Print results for verification
print(f"train_hog_real: {len(train_hog_real)} images, val_hog_real: {len(val_hog_real)} images")
print(f"train_hog_fake: {len(train_hog_fake)} images, val_hog_fake: {len(val_hog_fake)} images")


############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_labels_real = np.zeros(len(train_hog_real), dtype=int)
train_labels_fake = np.ones(len(train_hog_fake), dtype=int)

# Concatenate all training datasets into a single `train` variable
train = np.concatenate([train_hog_real, train_hog_fake], axis=0)
train_labels=np.concatenate([train_labels_real, train_labels_fake], axis=0)



test_labels_real = np.zeros(len(X_test_real), dtype=int)
test_labels_fake = np.ones(len(X_train_fake), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test = np.concatenate([X_test_real, X_train_fake], axis=0)
test_labels = np.concatenate([test_labels_real, test_labels_fake], axis=0)



val_labels_real = np.zeros(len(val_hog_real), dtype=int)
val_labels_fake = np.ones(len(val_hog_fake), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val = np.concatenate([val_hog_real, val_hog_fake], axis=0)
val_labels = np.concatenate([val_labels_real, val_labels_fake], axis=0)

# Print the results for verification
print(f"Total train: {len(train)} images")
print(f"Total test: {len(test)} images")
print(f"Total val: {len(val)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



Training data real shape: (6909, 160, 160, 3)
Testing real data shape: (2961, 160, 160, 3)
Training fake data shape: (7891, 160, 160, 3)
Testing  fake data shape: (3383, 160, 160, 3)
train_hog_real: 6218 images, val_hog_real: 691 images
train_hog_fake: 7101 images, val_hog_fake: 790 images
Total train: 13319 images
Total test: 10852 images
Total val: 1481 images
Train Labels: 13319 
Test Labels: 10852 
Val Labels: 1481 


In [3]:
print("=== DATA LOADING ===")
start = monitor.get_stats()

=== DATA LOADING ===


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the model
def build_xception_model(input_shape=(160, 160, 3)):
    base_model = tf.keras.applications.Xception(
        weights='imagenet',  # Pretrained on ImageNet
        include_top=False,   # Exclude the top classification layer
        input_shape=input_shape
    )
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary classification output
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Build the model
model = build_xception_model(input_shape=(160, 160, 3))



In [6]:

# Train model
history = model.fit(
    train, train_labels,
    validation_data=(val, val_labels),
    epochs=100,
    batch_size=32,
    verbose=1
)
test_loss, test_acc = model.evaluate(test, test_labels)

Epoch 1/100
417/417 [==============================] - 21s 38ms/step - loss: 1.7117 - accuracy: 0.5295 - val_loss: 0.7041 - val_accuracy: 0.5658
Epoch 2/100
417/417 [==============================] - 15s 36ms/step - loss: 0.7186 - accuracy: 0.5669 - val_loss: 0.6706 - val_accuracy: 0.5854
Epoch 3/100
417/417 [==============================] - 15s 37ms/step - loss: 0.6792 - accuracy: 0.5940 - val_loss: 0.6589 - val_accuracy: 0.6104
Epoch 4/100
417/417 [==============================] - 15s 36ms/step - loss: 0.6620 - accuracy: 0.5961 - val_loss: 0.6511 - val_accuracy: 0.6354
Epoch 5/100
417/417 [==============================] - 15s 37ms/step - loss: 0.6559 - accuracy: 0.6138 - val_loss: 0.6470 - val_accuracy: 0.6307
Epoch 6/100
417/417 [==============================] - 15s 36ms/step - loss: 0.6452 - accuracy: 0.6290 - val_loss: 0.6387 - val_accuracy: 0.6442
Epoch 7/100
417/417 [==============================] - 15s 36ms/step - loss: 0.6353 - accuracy: 0.6326 - val_loss: 0.6414 - val_ac

In [7]:
print(f'Test Accuracy of Celeb-df on Xception NET: {test_acc * 100:.2f}%')
from sklearn.metrics import classification_report
import numpy as np

# Get predictions (returns probabilities between 0-1)
y_pred_probs = model.predict(test)

# Convert probabilities to class labels (0 or 1)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

# Ensure test_labels is 1D array (remove extra dimensions if needed)
test_labels_flat = np.squeeze(test_labels)

# Print classification report
print(classification_report(test_labels_flat, y_pred_labels))

Test Accuracy of Celeb-df on Xception NET: 89.19%
340/340 [==============================] - 11s 30ms/step
              precision    recall  f1-score   support

           0       0.86      0.73      0.79      2961
           1       0.90      0.95      0.93      7891

    accuracy                           0.89     10852
   macro avg       0.88      0.84      0.86     10852
weighted avg       0.89      0.89      0.89     10852



In [ ]:

import tensorflow as tf

# After training the model
model.save('xception_160_celeb.h5')  # Saves the entire model to a file


In [9]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 5.4%
RAM Used: 4369.7 MB
Time Usage: 1566.1 s
GPU Memory Used: 1975.6 MB
Power Consumption: 93W


In [6]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('xception_160_celeb.h5')
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'Test Accuracy of Celeb-df on Xception NET: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference


340/340 [==============================] - 22s 40ms/step - loss: 0.3047 - accuracy: 0.8919
Test Accuracy of Celeb-df on Xception NET: 89.19%


#just testing

In [7]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 0.9%
RAM Used: 3425.2 MB
Time Usage: 40.9 s
GPU Memory Used: 892.8 MB
Power Consumption: 93W


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('xception_160_celeb.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on Xceptionnet from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 3s 31ms/step - loss: 3.7307 - accuracy: 0.5080

Test Accuracy of DFC on Xceptionnet from Celebdf: 50.80%
94/94 [==============================] - 3s 29ms/step
              precision    recall  f1-score   support

           0       0.50      0.94      0.66      1500
           1       0.56      0.08      0.14      1500

    accuracy                           0.51      3000
   macro avg       0.53      0.51      0.40      3000
weighted avg       0.53      0.51      0.40      3000



In [6]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Xceptionnet from Celebdf: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 5s 32ms/step - loss: 3.6365 - accuracy: 0.3733

Test Accuracy of FF++ on Xceptionnet from Celebdf: 37.33%
169/169 [==============================] - 5s 31ms/step
              precision    recall  f1-score   support

           0       0.29      0.89      0.44      1468
           1       0.81      0.18      0.29      3911

    accuracy                           0.37      5379
   macro avg       0.55      0.53      0.37      5379
weighted avg       0.67      0.37      0.33      5379



# DFC

In [2]:
import h5py
import numpy as np
# Open the HDF5 file in read mode
with h5py.File('D://thesis//dataset//deepfake dataset//resized_images.h5', 'r') as h5f:
    # Access each dataset
    celeb = np.array(h5f['celeb'])
    ffhq = np.array(h5f['ffhq'])
    gdwct = np.array(h5f['gdwct'])
    attgan = np.array(h5f['attgan'])
    stargan = np.array(h5f['stargan'])
    stylegan2 = np.array(h5f['stylegan2'])
    stylegan = np.array(h5f['stylegan'])

# Now, 'celeb', 'ffhq', etc., are NumPy arrays containing your datasets
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"ffhq shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"ffhq shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"ffhq shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"ffhq shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"ffhq shape: {stylegan.shape}, dtype: {stylegan.dtype}")
# Repeat for other datasets as needed
import cv2
# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images

celeb = resize_images(celeb, target_size=(160, 160))
ffhq = resize_images(ffhq, target_size=(160, 160))
gdwct = resize_images(gdwct, target_size=(160, 160))
attgan = resize_images(attgan, target_size=(160, 160))
stargan = resize_images(stargan, target_size=(160, 160))
stylegan = resize_images(stargan, target_size=(160, 160))
stylegan2 = resize_images(stylegan2, target_size=(160, 160))
import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(celeb)), 2500)  # Get 2500 random indices
celeb = celeb[random_indices]  # Select the random subse

import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(ffhq)), 2500)  # Get 2500 random indices
ffhq = ffhq[random_indices]  # Select the random subse
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"gdwct shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"attagan shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"stargan shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"stylegan2 shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"stylegan shape: {stylegan.shape}, dtype: {stylegan.dtype}")
import random
import numpy as np

def split_data(data, train_ratio=0.7):
    """
    Splits data into training and testing sets based on the specified ratio.

    Parameters:
        data (list or np.array): The dataset to split.
        train_ratio (float): The ratio of the data to include in the training set.

    Returns:
        tuple: Two datasets - train and test.
    """
    # Shuffle the data
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)

    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Split `celeb` into 70% train and 30% test
celeb_train_hog, celeb_test_hog = split_data(celeb, train_ratio=0.7)

# Split `ffhq` into 70% train and 30% test
ffhq_train_hog, ffhq_test_hog = split_data(ffhq, train_ratio=0.7)

# Split `attgan` into 70% train and 30% test
attgan_train_hog, attgan_test_hog = split_data(attgan, train_ratio=0.7)

# Split `stargan` into 70% train and 30% test
stargan_train_hog, stargan_test_hog = split_data(stargan, train_ratio=0.7)

# Split `gdwct` into 70% train and 30% test
gdwct_train_hog, gdwct_test_hog = split_data(gdwct, train_ratio=0.7)

# Split `stylegan2` into 70% train and 30% test_hog
stylegan2_train_hog, stylegan2_test_hog = split_data(stylegan2, train_ratio=0.7)

# Split `stylegan` into 70% train and 30% test_hog
stylegan_train_hog, stylegan_test_hog = split_data(stylegan, train_ratio=0.7)

# Convert to NumPy arrays if needed
celeb_train_hog, celeb_test_hog = np.array(celeb_train_hog), np.array(celeb_test_hog)
ffhq_train_hog, ffhq_test_hog = np.array(ffhq_train_hog), np.array(ffhq_test_hog)
attgan_train_hog, attgan_test_hog = np.array(attgan_train_hog), np.array(attgan_test_hog)
stargan_train_hog, stargan_test_hog = np.array(stargan_train_hog), np.array(stargan_test_hog)
gdwct_train_hog, gdwct_test_hog = np.array(gdwct_train_hog), np.array(gdwct_test_hog)
stylegan2_train_hog, stylegan2_test_hog = np.array(stylegan2_train_hog), np.array(stylegan2_test_hog)
stylegan_train_hog, stylegan_test_hog = np.array(stylegan_train_hog), np.array(stylegan_test_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_test: {len(celeb_test_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_test: {len(ffhq_test_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_test: {len(attgan_test_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_test: {len(stargan_test_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_test: {len(gdwct_test_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_test: {len(stylegan2_test_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_test: {len(stylegan_test_hog)} images")

########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
celeb_train_hog, celeb_val_hog = extract_validation(celeb_train_hog)
ffhq_train_hog, ffhq_val_hog = extract_validation(ffhq_train_hog)
attgan_train_hog, attgan_val_hog = extract_validation(attgan_train_hog)
stargan_train_hog, stargan_val_hog = extract_validation(stargan_train_hog)
gdwct_train_hog, gdwct_val_hog = extract_validation(gdwct_train_hog)
stylegan2_train_hog, stylegan2_val_hog = extract_validation(stylegan2_train_hog)
stylegan_train_hog, stylegan_val_hog = extract_validation(stylegan_train_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_val: {len(celeb_val_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_val: {len(ffhq_val_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_val: {len(attgan_val_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_val: {len(stargan_val_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_val: {len(gdwct_val_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_val: {len(stylegan2_val_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_val: {len(stylegan_val_hog)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


celeb_train_labels = np.zeros(len(celeb_train_hog), dtype=int)
ffhq_train_labels = np.zeros(len(ffhq_train_hog), dtype=int)
atta_train_labels = np.ones(len(attgan_train_hog), dtype=int)
star_train_labels = np.ones(len(stargan_train_hog), dtype=int)
gdwct_train_labels = np.ones(len(gdwct_train_hog), dtype=int)
stylegan2_train_labels = np.ones(len(stylegan2_train_hog), dtype=int)
stylegan_train_labels = np.ones(len(stylegan_train_hog), dtype=int)

# Concatenate all training datasets into a single `train` variable
train_hog = np.concatenate([celeb_train_hog, ffhq_train_hog, attgan_train_hog, stargan_train_hog, gdwct_train_hog, stylegan2_train_hog, stylegan_train_hog], axis=0)
train_labels=np.concatenate([celeb_train_labels, ffhq_train_labels, atta_train_labels, star_train_labels, gdwct_train_labels, stylegan2_train_labels,
                              stylegan_train_labels], axis=0)




celeb_test_labels = np.zeros(len(celeb_test_hog), dtype=int)
ffhq_test_labels = np.zeros(len(ffhq_test_hog), dtype=int)
atta_test_labels = np.ones(len(attgan_test_hog), dtype=int)
star_test_labels = np.ones(len(stargan_test_hog), dtype=int)
gdwct_test_labels = np.ones(len(gdwct_test_hog), dtype=int)
stylegan2_test_labels = np.ones(len(stylegan2_test_hog), dtype=int)
stylegan_test_labels = np.ones(len(stylegan_test_hog), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test_hog = np.concatenate([celeb_test_hog, ffhq_test_hog, attgan_test_hog, stargan_test_hog, gdwct_test_hog, stylegan2_test_hog, stylegan_test_hog], axis=0)
test_labels = np.concatenate([celeb_test_labels, ffhq_test_labels, atta_test_labels, star_test_labels, gdwct_test_labels, stylegan2_test_labels,
                        stylegan_test_labels], axis=0)




celeb_val_labels = np.zeros(len(celeb_val_hog), dtype=int)
ffhq_val_labels = np.zeros(len(ffhq_val_hog), dtype=int)
atta_val_labels = np.ones(len(attgan_val_hog), dtype=int)
star_val_labels = np.ones(len(stargan_val_hog), dtype=int)
gdwct_val_labels = np.ones(len(gdwct_val_hog), dtype=int)
stylegan2_val_labels = np.ones(len(stylegan2_val_hog), dtype=int)
stylegan_val_labels = np.ones(len(stylegan_val_hog), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val_hog = np.concatenate([celeb_val_hog, ffhq_val_hog, attgan_val_hog, stargan_val_hog, gdwct_val_hog, stylegan2_val_hog, stylegan_val_hog], axis=0)
val_labels = np.concatenate([celeb_val_labels, ffhq_val_labels, atta_val_labels, star_val_labels, gdwct_val_labels, stylegan2_val_labels,
                       stylegan_val_labels], axis=0)

# Print the results for verification
print(f"Total train: {len(train_hog)} images")
print(f"Total test: {len(test_hog)} images")
print(f"Total val: {len(val_hog)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



celeb shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
celeb shape: (2500, 160, 160, 3), dtype: uint8
ffhq shape: (2500, 160, 160, 3), dtype: uint8
gdwct shape: (1000, 160, 160, 3), dtype: uint8
attagan shape: (1000, 160, 160, 3), dtype: uint8
stargan shape: (1000, 160, 160, 3), dtype: uint8
stylegan2 shape: (1000, 160, 160, 3), dtype: uint8
stylegan shape: (1000, 160, 160, 3), dtype: uint8
celeb_train: 1750 images, celeb_test: 750 images
ffhq_train: 1750 images, ffhq_test: 750 images
attgan_train: 700 images, attgan_test: 300 images
stargan_train: 700 images, stargan_test: 300 images
gdwct_train: 700 images, gdwct_test: 300 images
stylegan2_train: 700 images, stylegan2_test: 300 images
stylegan_train: 700 images, stylegan

In [5]:

# Train model
history = model.fit(
    train_hog, train_labels,
    validation_data=(val_hog, val_labels),
    epochs=100,
    batch_size=32,
    verbose=1
)
test_loss, test_acc = model.evaluate(test_hog, test_labels)

Epoch 1/100
197/197 [==============================] - 13s 40ms/step - loss: 2.9255 - accuracy: 0.5354 - val_loss: 0.8598 - val_accuracy: 0.6143
Epoch 2/100
197/197 [==============================] - 7s 36ms/step - loss: 1.0222 - accuracy: 0.6054 - val_loss: 0.6267 - val_accuracy: 0.6543
Epoch 3/100
197/197 [==============================] - 7s 36ms/step - loss: 0.6906 - accuracy: 0.6394 - val_loss: 0.5801 - val_accuracy: 0.7086
Epoch 4/100
197/197 [==============================] - 7s 36ms/step - loss: 0.6117 - accuracy: 0.6787 - val_loss: 0.5364 - val_accuracy: 0.7371
Epoch 5/100
197/197 [==============================] - 7s 36ms/step - loss: 0.5799 - accuracy: 0.7041 - val_loss: 0.5560 - val_accuracy: 0.7543
Epoch 6/100
197/197 [==============================] - 7s 37ms/step - loss: 0.5515 - accuracy: 0.7229 - val_loss: 0.5094 - val_accuracy: 0.7743
Epoch 7/100
197/197 [==============================] - 7s 37ms/step - loss: 0.5358 - accuracy: 0.7306 - val_loss: 0.4955 - val_accuracy

In [7]:
print(f'Test Accuracy of DFC on Xception NET: {test_acc * 100:.2f}%')
from sklearn.metrics import classification_report
import numpy as np

# Get predictions (returns probabilities between 0-1)
y_pred_probs = model.predict(test_hog)

# Convert probabilities to class labels (0 or 1)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

# Ensure test_labels is 1D array (remove extra dimensions if needed)
test_labels_flat = np.squeeze(test_labels)

# Print classification report
print(classification_report(test_labels_flat, y_pred_labels))

Test Accuracy of DFC on Xception NET: 83.53%
94/94 [==============================] - 4s 33ms/step
              precision    recall  f1-score   support

           0       0.81      0.88      0.84      1500
           1       0.87      0.79      0.83      1500

    accuracy                           0.84      3000
   macro avg       0.84      0.84      0.83      3000
weighted avg       0.84      0.84      0.83      3000



In [8]:

import tensorflow as tf

# After training the model
model.save('xception_160_dfc.h5')  # Saves the entire model to a file


In [9]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 12.4%
RAM Used: 4343.3 MB
Time Usage: 843.3 s
GPU Memory Used: 829.2 MB
Power Consumption: 93W


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print("=== DATA LOADING ===")
start = monitor.get_stats()
# Load the saved model
model = tf.keras.models.load_model('xception_160_dfc.h5')
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'Test Accuracy of DFC on Xception NET: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE === only testing")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
94/94 [==============================] - 6s 35ms/step - loss: 0.7223 - accuracy: 0.8070
Test Accuracy of DFC on Xception NET: 80.70%

=== RESOURCE USAGE === only testing
CPU Usage: 5.4%
RAM Used: 3306.9 MB
Time Usage: 12.2 s
GPU Memory Used: 317.7 MB
Power Consumption: 93W


In [13]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('xception_160_dfc.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on Xceptionnet for DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 11s 32ms/step - loss: 1.9827 - accuracy: 0.4907

Test Accuracy of Celeb-df dataset on Xceptionnet for DFC: 49.07%
340/340 [==============================] - 12s 33ms/step
              precision    recall  f1-score   support

           0       0.28      0.55      0.37      2961
           1       0.74      0.47      0.57      7891

    accuracy                           0.49     10852
   macro avg       0.51      0.51      0.47     10852
weighted avg       0.61      0.49      0.52     10852



In [14]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'\nTest Accuracy of FF++ on Xceptionnet from DFC: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_ff)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels_ff, y_pred_binary))

169/169 [==============================] - 5s 32ms/step - loss: 2.6148 - accuracy: 0.4759

Test Accuracy of FF++ on Xceptionnet from DFC: 47.59%
169/169 [==============================] - 5s 30ms/step
              precision    recall  f1-score   support

           0       0.29      0.62      0.39      1468
           1       0.75      0.42      0.54      3911

    accuracy                           0.48      5379
   macro avg       0.52      0.52      0.47      5379
weighted avg       0.62      0.48      0.50      5379



In [ ]:
# ff++

In [2]:
import h5py

# Open the HDF5 file for reading
with h5py.File("D://thesis//dataset//ff++//images_ff++_orignal_processed.h5", "r") as h5f:
    org_seq_actor_array = h5f["ori_actor"][:]
    org_seq_youtube_array = h5f["ori_youtube"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake1_processed.h5", "r") as h5f:
    meni_seq_dfd_array = h5f["mni_dfd"][:]
    meni_seq_df_array = h5f["mni_df"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake2_processed.h5", "r") as h5f:
    meni_seq_f2f_array = h5f["mni_f2f"][:]
    meni_seq_fshifter_array = h5f["mni_fshifter"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake3_processed.h5", "r") as h5f:
    meni_seq_fswap_array = h5f["mni_fswap"][:]
    meni_seq_nt_array = h5f["mni_nt"][:]
print("Data loaded successfully from the HDF5 file.")
# Output the shape of the resulting array
print(f"Shape of the concatenated array: {org_seq_actor_array.shape}")
print(f"Shape of the concatenated array: {org_seq_youtube_array.shape}")

print(f"Shape of the concatenated array: {meni_seq_dfd_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_df_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fshifter_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_f2f_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fswap_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_nt_array.shape}")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ori_actor_train_real, ori_actor_test_real = train_test_split(org_seq_actor_array, test_size=0.3, random_state=42)
ori_youtube_train_real, ori_youtube_test_real = train_test_split(org_seq_youtube_array, test_size=0.3, random_state=42)
# Split the data into train (70%) and test (30%)
mni_dfd_train_fake, mni_dfd_test_fake = train_test_split(meni_seq_dfd_array, test_size=0.3, random_state=42)
mni_df_train_fake, mni_df_test_fake = train_test_split(meni_seq_df_array, test_size=0.3, random_state=42)
mni_f2f_train_fake, mni_f2f_test_fake = train_test_split(meni_seq_f2f_array, test_size=0.3, random_state=42)
mni_fshifter_train_fake, mni_fshifter_test_fake = train_test_split(meni_seq_fshifter_array, test_size=0.3, random_state=42)
mni_fswap_train_fake, mni_fswap_test_fake = train_test_split(meni_seq_fswap_array, test_size=0.3, random_state=42)
mni_nt_train_fake, mni_nt_test_fake = train_test_split(meni_seq_nt_array, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data ACTOR real shape:", ori_actor_train_real.shape, "Testing real data shape:", ori_actor_test_real.shape)
print("Training data Youtube real shape:", ori_youtube_train_real.shape, "Testing real data shape:", ori_youtube_test_real.shape)

print("Training data DFD fake shape:", mni_dfd_train_fake.shape, "Testing fake data shape:", mni_dfd_test_fake.shape)
print("Training data DF fake shape:", mni_df_train_fake.shape, "Testing fake data shape:", mni_df_test_fake.shape)
print("Training data f2f fake shape:", mni_f2f_train_fake.shape, "Testing fake data shape:", mni_f2f_test_fake.shape)
print("Training data fshifter fake shape:", mni_fshifter_train_fake.shape, "Testing fake data shape:", mni_fshifter_test_fake.shape)
print("Training data fswap fake shape:", mni_fswap_train_fake.shape, "Testing fake data shape:", mni_fswap_test_fake.shape)
print("Training data nt fake shape:", mni_nt_train_fake.shape, "Testing fake data shape:", mni_nt_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_ori_actor_real, val_hog_ori_actor_real = extract_validation(ori_actor_train_real)
train_hog_ori_youtube_real, val_hog_ori_youtube_real = extract_validation(ori_youtube_train_real)
train_hog_mni_dfd_fake, val_hog_mni_dfd_fake = extract_validation(mni_dfd_train_fake)
train_hog_mni_df_fake, val_hog_mni_df_fake = extract_validation(mni_df_train_fake)
train_hog_mni_f2f_fake, val_hog_mni_f2f_fake = extract_validation(mni_f2f_train_fake)
train_hog_mni_fshifter_fake, val_hog_mni_fshifter_fake = extract_validation(mni_fshifter_train_fake)
train_hog_mni_fswap_fake, val_hog_mni_fswap_fake = extract_validation(mni_fswap_train_fake)
train_hog_mni_nt_fake, val_hog_mni_nt_fake = extract_validation(mni_nt_train_fake)
# Print results for verification
print(f"train_ori_actor hog_real: {len(train_hog_ori_actor_real)} images, val_ori_actor hog_real: {len(val_hog_ori_actor_real)} images")
print(f"train_ ori_youtube hog_real: {len(train_hog_ori_youtube_real)} images, val_ ori_youtube hog_real: {len(val_hog_ori_youtube_real)} images")
print(f"train_hog_mni_dfd_fake: {len(train_hog_mni_dfd_fake)} images, val_hog_mni_dfd_fake: {len(val_hog_mni_dfd_fake)} images")
print(f"train_hog_mni_df_fake: {len(train_hog_mni_df_fake)} images, val_hog_mni_df_fake: {len(val_hog_mni_df_fake)} images")
print(f"train_hog_mni_f2f_fake: {len(train_hog_mni_f2f_fake)} images, val_hog_mni_f2f_fake: {len(val_hog_mni_f2f_fake)} images")
print(f"train_hog_mni_fshifter_fake: {len(train_hog_mni_fshifter_fake)} images, val_hog_mni_fshifter_fake: {len(val_hog_mni_fshifter_fake)} images")
print(f"train_hog_mni_fswap_fake: {len(train_hog_mni_fswap_fake)} images, val_hog_mni_fswap_fake: {len(val_hog_mni_fswap_fake)} images")
print(f"train_hog_mni_nt_fake: {len(train_hog_mni_nt_fake)} images, val_hog_mni_nt_fake: {len(val_hog_mni_nt_fake)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_ori_actor_labels_real = np.zeros(len(train_hog_ori_actor_real), dtype=int)
train_ori_youtube_labels_real = np.zeros(len(train_hog_ori_youtube_real), dtype=int)
train_mni_dfd_labels_fake = np.ones(len(train_hog_mni_dfd_fake), dtype=int)
train_mni_df_labels_fake = np.ones(len(train_hog_mni_df_fake), dtype=int)
train_mni_f2f_labels_fake = np.ones(len(train_hog_mni_f2f_fake), dtype=int)
train_mni_fshifter_labels_fake = np.ones(len(train_hog_mni_fshifter_fake), dtype=int)
train_mni_fswap_labels_fake = np.ones(len(train_hog_mni_fswap_fake), dtype=int)
train_mni_nt_labels_fake = np.ones(len(train_hog_mni_nt_fake), dtype=int)

test_ori_actor_labels_real = np.zeros(len(ori_actor_test_real), dtype=int)
test_ori_youtube_labels_real = np.zeros(len(ori_youtube_test_real), dtype=int)
test_mni_dfd_labels_fake = np.ones(len(mni_dfd_test_fake), dtype=int)
test_mni_df_labels_fake = np.ones(len(mni_df_test_fake), dtype=int)
test_mni_f2f_labels_fake = np.ones(len(mni_f2f_test_fake), dtype=int)
test_mni_fshifter_labels_fake = np.ones(len(mni_fshifter_test_fake), dtype=int)
test_mni_fswap_labels_fake = np.ones(len(mni_fswap_test_fake), dtype=int)
test_mni_nt_labels_fake = np.ones(len(mni_nt_test_fake), dtype=int)


val_ori_actor_labels_real = np.zeros(len(val_hog_ori_actor_real), dtype=int)
val_ori_youtube_labels_real = np.zeros(len(val_hog_ori_youtube_real), dtype=int)
val_mni_dfd_labels_fake = np.ones(len(val_hog_mni_dfd_fake), dtype=int)
val_mni_df_labels_fake = np.ones(len(val_hog_mni_df_fake), dtype=int)
val_mni_f2f_labels_fake = np.ones(len(val_hog_mni_f2f_fake), dtype=int)
val_mni_fshifter_labels_fake = np.ones(len(val_hog_mni_fshifter_fake), dtype=int)
val_mni_fswap_labels_fake = np.ones(len(val_hog_mni_fswap_fake), dtype=int)
val_mni_nt_labels_fake = np.ones(len(val_hog_mni_nt_fake), dtype=int)
##################################################################################################################

# Concatenate all training datasets into a single `train` variable
train_ff = np.concatenate([train_hog_ori_actor_real, train_hog_ori_youtube_real,train_hog_mni_dfd_fake,train_hog_mni_df_fake,
                            train_hog_mni_f2f_fake,train_hog_mni_fshifter_fake,train_hog_mni_fswap_fake,train_hog_mni_nt_fake], axis=0)

train_labels_ff = np.concatenate([train_ori_actor_labels_real, train_ori_youtube_labels_real,train_mni_dfd_labels_fake,train_mni_df_labels_fake,
                            train_mni_f2f_labels_fake,train_mni_fshifter_labels_fake,train_mni_fswap_labels_fake,train_mni_nt_labels_fake], axis=0)

# Concatenate all validation datasets into a single `val` variable
val_ff = np.concatenate([val_hog_ori_actor_real, val_hog_ori_youtube_real, val_hog_mni_dfd_fake, val_hog_mni_df_fake,
                            val_hog_mni_f2f_fake, val_hog_mni_fshifter_fake, val_hog_mni_fswap_fake, val_hog_mni_nt_fake], axis=0)
val_labels_ff = np.concatenate([val_ori_actor_labels_real, val_ori_youtube_labels_real, val_mni_dfd_labels_fake, val_mni_df_labels_fake,
                            val_mni_f2f_labels_fake, val_mni_fshifter_labels_fake, val_mni_fswap_labels_fake, val_mni_nt_labels_fake], axis=0)
# Concatenate all testing datasets into a single `test` variable
test_ff = np.concatenate([ori_actor_test_real, ori_youtube_test_real, mni_dfd_test_fake,
                           mni_df_test_fake, mni_f2f_test_fake, mni_fshifter_test_fake,
                           mni_fswap_test_fake, mni_nt_test_fake], axis=0)
test_labels_ff = np.concatenate([test_ori_actor_labels_real, test_ori_youtube_labels_real, test_mni_dfd_labels_fake, test_mni_df_labels_fake,
                            test_mni_f2f_labels_fake, test_mni_fshifter_labels_fake, test_mni_fswap_labels_fake, test_mni_nt_labels_fake], axis=0)


# Print the results for verification
# Print the results for verification
print(f"Total train: {len(train_ff)} images, and shape:{train_ff.shape}")
print(f"Total test: {len(test_ff)} images, and shape:{test_ff.shape}")
print(f"Total val: {len(val_ff)} images, and shape:{val_ff.shape}")


# Print results for verification
print(f"Train Labels: {len(train_labels_ff)} ")
print(f"Test Labels: {len(test_labels_ff)} ")
print(f"Val Labels: {len(val_labels_ff)} ")





Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Shape of the concatenated array: (2808, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (3299, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Training data ACTOR real shape: (1965, 160, 160, 3) Testing real data shape: (843, 160, 160, 3)
Training data Youtube real shape: (1458, 160, 160, 3) Testing real data shape: (625, 160, 160, 3)
Training data DFD fake shape: (2309, 160, 160, 3) Testing fake data shape: (990, 160, 160, 3)
Training data DF fake shape: (1458, 160, 160, 3) Testing fake data shape: (625, 160, 160, 3)
Training data f

In [10]:

# Train model
history = model.fit(
    train_ff, train_labels_ff,
    validation_data=(val_ff, val_labels_ff),
    epochs=100,
    batch_size=32,
    verbose=1
)
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)


Epoch 1/100
353/353 [==============================] - 19s 39ms/step - loss: 1.9686 - accuracy: 0.6395 - val_loss: 0.6430 - val_accuracy: 0.7229
Epoch 2/100
353/353 [==============================] - 13s 36ms/step - loss: 0.6698 - accuracy: 0.7009 - val_loss: 0.5751 - val_accuracy: 0.7333
Epoch 3/100
353/353 [==============================] - 13s 37ms/step - loss: 0.5931 - accuracy: 0.7250 - val_loss: 0.5661 - val_accuracy: 0.7373
Epoch 4/100
353/353 [==============================] - 13s 37ms/step - loss: 0.5742 - accuracy: 0.7325 - val_loss: 0.5616 - val_accuracy: 0.7420
Epoch 5/100
353/353 [==============================] - 13s 37ms/step - loss: 0.5653 - accuracy: 0.7351 - val_loss: 0.5590 - val_accuracy: 0.7492
Epoch 6/100
353/353 [==============================] - 13s 37ms/step - loss: 0.5646 - accuracy: 0.7368 - val_loss: 0.5628 - val_accuracy: 0.7404
Epoch 7/100
353/353 [==============================] - 13s 37ms/step - loss: 0.5589 - accuracy: 0.7389 - val_loss: 0.5607 - val_ac

In [13]:
print(f'Test Accuracy of FF++ on Xception NET: {test_acc * 100:.2f}%')
from sklearn.metrics import classification_report
import numpy as np

# Get predictions (returns probabilities between 0-1)
y_pred_probs = model.predict(test_ff)

# Convert probabilities to class labels (0 or 1)
y_pred_labels = (y_pred_probs > 0.5).astype(int)

# Ensure test_labels is 1D array (remove extra dimensions if needed)
test_labels_flat = np.squeeze(test_labels_ff)

# Print classification report
print(classification_report(test_labels_flat, y_pred_labels))

Test Accuracy of FF++ on Xception NET: 73.77%
169/169 [==============================] - 5s 30ms/step
              precision    recall  f1-score   support

           0       0.53      0.34      0.41      1468
           1       0.78      0.89      0.83      3911

    accuracy                           0.74      5379
   macro avg       0.66      0.61      0.62      5379
weighted avg       0.71      0.74      0.72      5379



In [14]:

import tensorflow as tf

# After training the model
model.save('xception_160_ff.h5')  # Saves the entire model to a file


In [15]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 7.2%
RAM Used: 2635.0 MB
Time Usage: 1462.5 s
GPU Memory Used: 490.7 MB
Power Consumption: 93W


In [3]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
print("=== DATA LOADING ===")
start = monitor.get_stats()
model = tf.keras.models.load_model('xception_160_ff.h5')
test_loss, test_acc = model.evaluate(test_ff, test_labels_ff)
print(f'Test Accuracy of FF on Xception NET: {test_acc * 100:.2f}%')
# You can now use the model for testing or inference
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

=== DATA LOADING ===
169/169 [==============================] - 9s 34ms/step - loss: 0.6684 - accuracy: 0.7375
Test Accuracy of FF on Xception NET: 73.75%

=== RESOURCE USAGE ===
CPU Usage: 6.6%
RAM Used: 3263.3 MB
Time Usage: 15.3 s
GPU Memory Used: 492.0 MB
Power Consumption: 135W


In [16]:
from tensorflow.keras.models import load_model
import tensorflow as tf
# Load the saved model
model = tf.keras.models.load_model('xception_160_ff.h5')
# Evaluate on test set
test_loss, test_acc = model.evaluate(test, test_labels)
print(f'\nTest Accuracy of Celeb-df dataset on Xceptionnet for FF++: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))


340/340 [==============================] - 12s 34ms/step - loss: 0.7560 - accuracy: 0.7195

Test Accuracy of Celeb-df dataset on Xceptionnet for FF++: 71.95%
340/340 [==============================] - 11s 31ms/step
              precision    recall  f1-score   support

           0       0.38      0.05      0.08      2961
           1       0.73      0.97      0.83      7891

    accuracy                           0.72     10852
   macro avg       0.56      0.51      0.46     10852
weighted avg       0.64      0.72      0.63     10852



In [19]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_hog, test_labels)
print(f'\nTest Accuracy of DFC on Xceptionnet from FF++: {test_acc*100:.2f}%')

# Classification report
y_pred = model.predict(test_hog)
y_pred_binary = (y_pred > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred_binary))

94/94 [==============================] - 3s 34ms/step - loss: 0.9595 - accuracy: 0.5447

Test Accuracy of DFC on Xceptionnet from FF++: 54.47%
94/94 [==============================] - 3s 30ms/step
              precision    recall  f1-score   support

           0       0.59      0.29      0.39      1500
           1       0.53      0.80      0.64      1500

    accuracy                           0.54      3000
   macro avg       0.56      0.54      0.51      3000
weighted avg       0.56      0.54      0.51      3000

